# Wine Map Notebook
    
    
This notebook is for prototyping a winemap that will empower users to travel to destinations with good wine.

First of all we will import all the modules we need.


In [3]:
import pyspark
from pyspark.sql import SparkSession, SQLContext
from pyspark.context import SparkContext
from pyspark.sql.functions import mean, desc
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

Then we will create or get a spark cluster to do our analysis with.

In [4]:
sparkSession = SparkSession.builder.master("local").getOrCreate()

Here we create a dataframe to hold the data. This works well for data that is organised into columns, which our data is!
    
Tip: The data is loaded into memory when you make a dataframe like this so for prototyping you may want to cut out unnecessary columns (like description) and not use them for this.

In [5]:
df = (sparkSession.read
        .format("com.databricks.spark.csv")
        .option("header", "true")
        .option("inferSchema", "true")
        .load("wine_no_des.csv"))

Now we want to so some analysis with Apache Spark So we take the columns country and points and we groupby country. We then want to take the mean of these values for each country and put them into a list.


In [8]:
table = df.select('country','points').groupBy('country').agg(mean('points')).orderBy('avg(points)',ascending=False)
countryCols = table.select('country').collect()
countries = list()
for country in countryCols:
    countries.append(str(country[0]))
pointCols = table.select('avg(points)').collect()
points = list()
for point in pointCols:
    points.append(point[0])

Then we want to take these results and add them into a map.



In [12]:
data =  dict(type = 'choropleth',
        locationmode='country names',
        locations = countries,
        colorscale='Jet',
        z = points,
        colorbar = {'title': 'Average Rating'}
        )
layout = dict(geo = {'scope':'world'})
choromap = go.Figure(data = [data],layout = layout)
iplot(choromap)